### Funções

**Dicionário com nome das imagens e respectivas dimensões WxH**

In [1]:
def get_image_shapes(path_to_images):
    images_shapes = {}

    images_folder = os.listdir(path_to_images)

    for image_file_name in images_folder:
        #print(image_file_name)
        image = cv2.imread(path_to_images+image_file_name)
        try:
            height, width, channels = image.shape
            #print(f'\t{type(width)} {type(height)}')
            images_shapes[image_file_name[:-4]] = [width, height]
        except:
            print('no shape info.')
            return 0
    
    return images_shapes

**Conversão das datas detectadas em cada imagem: do formato yolo para coordenadas x1,y1,x2,y2 (max e min)**

In [2]:
def convert_yolo_labels_to_bbox(path_to_labels, image_shapes,conf_threshold=0.25): #vai retornar um dicionario 

    labels_bbox_dictionary = {}

    labels_folder = os.listdir(path_to_labels)
    #print(labels_folder)

    for label_file_name in labels_folder:
        # print(label_file_name)
        if label_file_name[-4:] == '.txt':
            with open(path_to_labels+label_file_name,'r') as label_file:
                labels = label_file.readlines()
                list_bbox = []
                for label_line in labels:
                    if label_line[0] == '1':

                        # print(f'\t{label_line}')
                        yolo_label = label_line.split(' ')[1:]
                        
                        confidence = -1
                        # print(f'\t\tlabel len: {len(yolo_label)}')
                        if len(yolo_label) == 5:
                            confidence = float(yolo_label[4])

                        if confidence == -1 or confidence >= conf_threshold:
                            
                            # print(f'\t\t{confidence}')
                            
                            width, height = image_shapes[label_file_name[:-4]]
                            #print(f'{width} {height}')
                            
                            x_center, y_center, w, h = float(yolo_label[0]), float(yolo_label[1]), float(yolo_label[2]), float(yolo_label[3])
                            # print(f'\t\t{x_center}, {y_center}, {w}, {h}')

                            x1 = round((x_center-w/2)*width)
                            y1 = round((y_center-h/2)*height)
                            x2 = round((x_center+w/2)*width)
                            y2 = round((y_center+h/2)*height)
                            # print(f'{x1}, {y1}, {x2}, {y2}')

                            list_bbox.append([x1,y1,x2,y2])
                        
                labels_bbox_dictionary[label_file_name[:-4]] = list_bbox

    return labels_bbox_dictionary

**Cálculos das métricas**

In [3]:
#Funções utilizadas para validar os resultados de Mean IoU e Mean Dice feitos dentro da função 'calculate_precision_recall'
def calculate_dice_score(boxA, boxB):
    # Calculate intersection area
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    # Calculate areas of each box
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    
    # Calculate Dice score
    dice_score = (2 * interArea) / (areaA + areaB) if (areaA + areaB) > 0 else 0
    return dice_score

def calculate_IoU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1) # utilizando +1 pois as coordenadas utilizadas estão em pixels e +1 inclui o pixel inicial no intervalo
    
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def calculate_mean_iou(detections, ground_truths, iou_threshold=0.5):
    total_iou = 0
    matched_detections = 0
    
    for img_id in detections.keys():
        # Get predictions and ground truths for the image
        predicted_boxes = detections[img_id]  # List of predicted bounding boxes
        gt_boxes = ground_truths[img_id]      # List of ground truth bounding boxes
        
        for pred_box in predicted_boxes:
            best_iou = 0
            for gt_box in gt_boxes:
                iou = calculate_IoU(pred_box, gt_box)
                if iou > best_iou:
                    best_iou = iou
            
            # If best IoU meets the threshold, count it as a match (True Positive)
            if best_iou >= iou_threshold:
                total_iou += best_iou
                matched_detections += 1
    
    # Calculate the mean IoU
    mean_iou = total_iou / matched_detections if matched_detections > 0 else 0
    return mean_iou

def calculate_mean_dice(detections, ground_truths, iou_threshold=0.5):
    total_dice = 0
    matched_detections = 0
    
    for img_id in detections.keys():
        predicted_boxes = detections[img_id]  # List of predicted bounding boxes
        gt_boxes = ground_truths[img_id]      # List of ground truth bounding boxes
        
        for pred_box in predicted_boxes:
            best_dice = 0
            for gt_box in gt_boxes:
                iou = calculate_IoU(pred_box, gt_box)
                
                if iou >= iou_threshold:  # If IoU meets the threshold, calculate Dice
                    dice = calculate_dice_score(pred_box, gt_box)
                    if dice > best_dice:
                        best_dice = dice
            
            # If the predicted box has a matching ground truth, count it as True Positive
            if best_dice > 0:
                total_dice += best_dice
                matched_detections += 1
    
    # Calculate the mean Dice score
    mean_dice = total_dice / matched_detections if matched_detections > 0 else 0
    return mean_dice

In [4]:

# Função para calcular todas das métricas entre 2 dicionarioa (GT vs yolo) (GT vs framework)
def calculate_precision_recall(labels_GT, labels_predicted, iou_threashhold=0.5):
    measures = {'TP':0, 'FN':0, 'FP':0,'TN':0}

    total_iou = 0
    total_dice = 0
    for img_name in labels_GT:

        if img_name not in labels_predicted:
            measures['FN']+= len(labels_GT[img_name])
            # print(f'imagem {img_name} não teve bbox detectada, mas existem {len(labels_GT[img_name])}')
            continue
        
        gt_bbox_list = labels_GT[img_name]
        pred_bbox_list = labels_predicted[img_name]

        matched_gt = []

        for pred_box in pred_bbox_list:
            best_iou = 0
            best_dice = 0
            best_gt_box = None 

            for gt_box in gt_bbox_list:
                if gt_box not in matched_gt:
                    iou = calculate_IoU(gt_box, pred_box)

                    if iou > best_iou:
                        best_iou = iou 
                        best_gt_box = gt_box

                        dice = calculate_dice_score(gt_box,pred_box)
                        if dice > best_dice:
                            best_dice = dice

            if best_iou >= iou_threashhold:
                measures['TP']+=1
                matched_gt.append(best_gt_box)
                total_iou += best_iou
                total_dice += best_dice
            else:
                measures['FP']+=1
        
        measures['FN'] += len(gt_bbox_list) - len(matched_gt)

    TP = measures['TP']
    FP = measures['FP']
    FN = measures['FN']
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0     # aka True Positive Rate
    # Não é possível calcular False Positive Rate pois utiliza valores de True Negative. Influenciado pale tamanho da classe 

    accuracy = TP / (TP + FP + FN)

    f1 = 2 * (precision * recall) / (precision + recall)
    
    mean_iou = total_iou / TP if TP > 0 else 0
    mean_dice = total_dice / TP if TP > 0 else 0

    return accuracy, precision, recall, f1, measures, mean_iou, mean_dice

### Variáveis para diretórios

In [ ]:
import os, cv2

# Diretório dos Labels Ground Truth do dataset
TEST_FOLDER = '/home/landreotti/tcc/expiry-date-detection-YOLO/datasets/Exp-Dates.v4i.yolov5pytorch/test/'
TEST_IMAGES_FOLDER = '/home/landreotti/tcc/expiry-date-detection-YOLO/datasets/Exp-Dates.v4i.yolov5pytorch/test/images/'
TEST_LABELS_FOLDER = '/home/landreotti/tcc/expiry-date-detection-YOLO/datasets/Exp-Dates.v4i.yolov5pytorch/test/labels/'

########################## 

framework_predictions = '/home/landreotti/tcc/expiry-date-detection-YOLO/resultados-tcc/test/Framework/2024-11-13_experimento_datasetv4/dates_detected_bbox/'

yolo_predictions_300_IoU60_Conf398 = '/home/landreotti/tcc/expiry-date-detection-YOLO/resultados-tcc/test/300/test_detect/exp47/labels/'

yolo_predictions_300_DataAug_IoU60_Conf368 = '/home/landreotti/tcc/expiry-date-detection-YOLO/resultados-tcc/test/300+DataAug/test_detect/exp36/labels/'

yolo_test_val_conf_thres_300_IoU60_Conf398 = '/home/landreotti/tcc/expiry-date-detection-YOLO/resultados-tcc/test/300/test_val/labels/'
yolo_test_val_conf_thres_300_DataAug_IoU60_Conf368= "/home/landreotti/tcc/expiry-date-detection-YOLO/resultados-tcc/test/300+DataAug/test_val/labels/"

##########################

### Script geral

**Armazenando labels ground-truth e preditos (yolo & framework) em dicionários.**
- As chaves dos dicionários são os nomes das imagens e valor é a lista de labels (em coordenadas x1,y1,x2,y2) 

In [6]:
def get_results(predicted_labels_folder, result_name, iou_threashold=0.5, conf_threasholf=0.25):
    
    print(f'\n... start results for >> {result_name} <<')

    images_shapes = get_image_shapes(TEST_IMAGES_FOLDER)

    labels_ground_truth = convert_yolo_labels_to_bbox(TEST_LABELS_FOLDER, images_shapes)
    labels_predicted = convert_yolo_labels_to_bbox(predicted_labels_folder, images_shapes, conf_threasholf)
    
    # print("\n============ Labels Ground Truth ============")
    # print(f'{len(labels_ground_truth)} \t total de imagens com datas a serem detectadas') #total de imagens com datas a serem detectadas

    # total_labels_gt = 0
    # for img_name in labels_ground_truth:
    #     total_labels_gt += len(labels_ground_truth[img_name])
    # print(f"{total_labels_gt} \t total de datas a serem detectadas")

    # print(f"\n============ Detected by {result_name} ============")

    # print(f"{len(labels_predicted)} \t total de imagens onde datas foram detectadas") #total de imagens onde datas foram detectadas

    # total_labels_predicted = 0
    # for img_name in labels_predicted:
    #     total_labels_predicted += len(labels_predicted[img_name])
    # print(f"{total_labels_predicted} \t total de datas detectadas")


    # print('\n\n\t... calculate iou, dice, mean_iou, mean_dice')
    # print(f'\t{result_name}')
    # print(f'\t\t{calculate_mean_iou(labels_predicted, labels_ground_truth)}')
    # print(f'\t\t{calculate_mean_dice(labels_predicted, labels_ground_truth)}')
    # print('--------------------------------------------')

    print('\n.. calculate all (accuracy, precision, recall, f1, ...)')
    accuracy, precision, recall, f1, measures, mean_iou, mean_dice = calculate_precision_recall(labels_ground_truth, labels_predicted, iou_threashold)
    print(f"{'Accuracy':>10s}\t{'Precision':>10s}\t{'Recall':>10s}\t{'F1':>10s}\t{'Mean Iou':>10s}\t{'Mean Dice':>10s}")
    print(f"{accuracy*100:10.3f}\t{precision*100:10.3f}\t{recall*100:10.3f}\t{f1*100:10.3f}\t{mean_iou*100:10.3f}\t {mean_dice*100:10.3f}")
    print('----------------------------------------------------------')

In [9]:
get_results(framework_predictions,'framework_predictions',0.60)
get_results(yolo_predictions_300_IoU60_Conf398,'yolo_predictions_300_IoU60_Conf398',0.6,0.398)
get_results(yolo_predictions_300_DataAug_IoU60_Conf368,'yolo_predictions_300_DataAug_IoU60_Conf368',0.6,0.368)
get_results(yolo_test_val_conf_thres_300_DataAug_IoU60_Conf368,'yolo_test_val_conf_thres_300_DataAug_IoU60_Conf368',0.6,0.368)


... start results for >> framework_predictions <<

.. calculate all (accuracy, precision, recall, f1, ...)
  Accuracy	 Precision	    Recall	        F1	  Mean Iou	 Mean Dice
    37.931	    62.857	    48.889	    55.000	    75.981	     85.753
----------------------------------------------------------

... start results for >> yolo_predictions_300_IoU60_Conf398 <<

.. calculate all (accuracy, precision, recall, f1, ...)
  Accuracy	 Precision	    Recall	        F1	  Mean Iou	 Mean Dice
    53.097	    72.289	    66.667	    69.364	    79.425	     88.060
----------------------------------------------------------

... start results for >> yolo_predictions_300_DataAug_IoU60_Conf368 <<

.. calculate all (accuracy, precision, recall, f1, ...)
  Accuracy	 Precision	    Recall	        F1	  Mean Iou	 Mean Dice
    63.964	    77.174	    78.889	    78.022	    82.037	     89.698
----------------------------------------------------------

... start results for >> yolo_test_val_conf_thres_300_DataAug_IoU